In [ ]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Constants
DATA_FOLDER = './data/fma_metadata'


In [ ]:
# Load datasets into pandas dataframes
# TODO: Specify dtypes
echonest_path = '{data}/{name}'.format(data=DATA_FOLDER,
                                       name='echonest.csv')
echonest_df = pd.read_csv(echonest_path, header=3)

features_path = '{data}/{name}'.format(data=DATA_FOLDER,
                                       name='features.csv')
features_df = pd.read_csv(features_path, header=4)

genres_path = '{data}/{name}'.format(data=DATA_FOLDER,
                                     name='genres.csv')
genres_dtypes = {'genre_id': int, '#tracks': int, 
                 'parent': int, 'top_level': int}
genres_df = pd.read_csv(genres_path, dtype=genres_dtypes)

tracks_path = '{data}/{name}'.format(data=DATA_FOLDER,
                                         name='tracks.csv')
tracks_df = pd.read_csv(tracks_path)


In [ ]:
genres_df.head()